In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from tensorflow.keras.models import load_model
model = load_model('../model_saved/b_tag_model.h5')

2023-05-19 06:08:27.212404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102cuda/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102cuda/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102cuda/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102cuda/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102cuda/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/c

In [3]:
import hls4ml
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']

In [ ]:
%%capture

int_bit = 10
i = 10
reuse = 4
precision = 'ap_fixed<{},{}>'.format((int_bit+i), int_bit)
fix_int_precision = 'ap_fixed<{},{}>'.format((16+i), 14)
dirc = 'runs/B_tagging_run/r4_{}int_{}frac'.format(int_bit, i)
#First, the baseline model
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = precision
hls_config['Model']['ReuseFactor'] = reuse
hls_config['Model']['Strategy'] = 'Resource'

for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Precision'] = precision
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = reuse
    hls_config['LayerName'][Layer]['weight'] = precision
    hls_config['LayerName'][Layer]['scale'] = precision
    hls_config['LayerName'][Layer]['bias'] = precision
    hls_config['LayerName'][Layer]['accum_t'] = fix_int_precision
    if 'multi_head_attention' in Layer:
        hls_config['LayerName'][Layer]['table_t'] = fix_int_precision
        hls_config['LayerName'][Layer]['inv_range'] = 256
        hls_config['LayerName'][Layer]['exp_range'] = 4
        hls_config['LayerName'][Layer]['table_size'] = 1024
    if 'dense' in Layer:
        hls_config['LayerName'][Layer]['table_t'] = fix_int_precision
        hls_config['LayerName'][Layer]['inv_range'] = 256
        hls_config['LayerName'][Layer]['table_size'] = 1024
        hls_config['LayerName'][Layer]['exp_range'] = 4

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_parallel' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = dirc
cfg['Part'] = 'xcvu13p-fhga2104-2L-e'

hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()
hls_model.build(csim=False, synth=True, vsynth=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
  **** IP Build 2902112 on Wed May 27 22:43:36 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2020.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2020.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'zhjiang' on host 'olivb-16.cern.ch' (Linux_x86_64 version 3.10.0-1160.88.1.el7.x86_64) on Fri May 19 06:08:45 CEST 2023
INFO: [HLS 200-10] In directory '/data/hlssynt-users/zhjiang/runs/B_tagging_run/r2_10int_10frac'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/data/hlssynt-users/zhjiang/runs/B_tagging_run/r2_10int_10frac/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 

INFO: [HLS 200-111] Finished Linking Time (s): cpu = 00:00:55 ; elapsed = 00:00:58 . Memory (MB): peak = 1630.246 ; gain = 1103.887 ; free physical = 35602 ; free virtual = 77288
INFO: [HLS 200-111] Finished Checking Pragmas Time (s): cpu = 00:00:55 ; elapsed = 00:00:58 . Memory (MB): peak = 1630.246 ; gain = 1103.887 ; free physical = 35601 ; free virtual = 77288
INFO: [HLS 200-10] Starting code transformations ...
INFO: [XFORM 203-603] Inlining function 'nnet::product::mult<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0> >::limit' into 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_1>' (firmware/nnet_utils/nnet_dense_latency.h:56).
INFO: [XFORM 203-603] Inlining function 'nnet::product::mult<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0> >::limit' into 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_

INFO: [XFORM 203-603] Inlining function 'nnet::dense_resource_rf_leq_nin<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config18>' into 'nnet::dense_resource<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config18>' (firmware/nnet_utils/nnet_dense_resource.h:276).
INFO: [XFORM 203-603] Inlining function 'nnet::dense_seq<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config18>' into 'nnet::dense<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config18>' (firmware/nnet_utils/nnet_dense.h:47).
INFO: [XFORM 203-603] Inlining function 'nnet::dense<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config18>' into 'myproject' (firmware/myproject.cpp:151).
INFO: [XFORM 203-603] Inlining function 'nnet::dense_resource_rf_leq_nin<ap_fixed<20, 10, (a

INFO: [HLS 200-111] Finished Checking Synthesizability Time (s): cpu = 00:01:50 ; elapsed = 00:01:54 . Memory (MB): peak = 1630.246 ; gain = 1103.887 ; free physical = 35063 ; free virtual = 76820
INFO: [XFORM 203-1101] Packing variable 'v_proj.V.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:293) into a 640-bit variable.
INFO: [XFORM 203-1101] Packing variable 'datav_pack.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:131) into a 640-bit variable.
INFO: [XFORM 203-1101] Packing variable 'q_proj.V.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:291) into a 640-bit variable.
INFO: [XFORM 203-1101] Packing variable 'k_proj.V.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:292) into a 640-bit variable.
INFO: [XFORM 203-1101] Packing variable 'datak_pack.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:76) into a 640-bit variable.
INFO: [XFORM 203-1101] Packing variable 'dataq_pack.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:76) into a 640-bit

INFO: [XFORM 203-502] Unrolling all loops for pipelining in function 'nnet::relu<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, relu_config12>' (firmware/nnet_utils/nnet_activation.h:68:39).
INFO: [XFORM 203-502] Unrolling all loops for pipelining in function 'nnet::add<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config10>' (firmware/nnet_utils/nnet_merge.h:62:38).
INFO: [XFORM 203-502] Unrolling all sub-loops inside loop 'output_dense' (firmware/nnet_utils/nnet_multiheadattention.h:242) in function 'nnet::dense_out<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config9>' for pipelining.
INFO: [XFORM 203-502] Unrolling all loops for pipelining in function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config9_2>' 

...

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_2>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fix

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_2>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_la

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_2>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nn

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_1>'(firmware/nnet_utils/nnet_dense_latency.h:35:1) to 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_1>.0' by setting 'weights[383].V' to 'key_weight2.V.383'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_1>'(firmware/nnet_utils/nnet_dense_latency.h:35:1) to 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_1>.1' by setting 'weights[383].V' to 'query_weight2.V.383'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_1>'(

INFO: [XFORM 203-712] Applying dataflow to function 'nnet::multiheadattention<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>', detected/extracted 12 process function(s): 
	 'nnet::multiheadattention<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>.entry97699887'
	 'nnet::data_prep<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>663'
	 'nnet::data_prep<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>664'
	 'nnet::data_prep<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>665'
	 'nnet::data_prep<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>666'
	 'nnet::lin_projection<ap_fixed<20, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<20, 10, (ap